"""<br>
    @Author: Deven Gupta<br>
    @Date: 12-10-2024<br>
    @Last Modified by: Deven Gupta<br>
    @Last Modified time: 12-10-2024<br>
    @Title: Task1- Using Email for (Summarization and Transformation) using Google Gemini.<br>
<br>
"""<br>

## Summarization and Translation

Installing library for gemini -- `! pip install google-generativeai ` 

In [4]:
import os
import google.generativeai as genai
import csv

# Configure the API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [5]:
def process_emails(input_file, output_file):
    """
    Description:
        Processes emails from an input text file, summarizes the body, translates the summary to French, 
        and writes the results to a CSV file.

    Parameters:
        input_file (str): Path to the input text file containing emails in a specific format.
        output_file (str): Path to the output CSV file where results will be stored.

    Returns: None
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        emails = f.read().strip().split('\n\n')  # Split by double newlines

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['sender', 'receiver', 'summarized_body', 'translated_body']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for email in emails:
            lines = email.strip().split('\n')

            # Check if we have at least the expected lines
            if len(lines) < 4:
                print(f"Skipping malformed email:\n{email}\n")
                continue

            subject = lines[0].replace('Subject: ', '').strip()
            sender = lines[1].replace('From: ', '').strip()
            receiver = lines[2].replace('To: ', '').strip()
            body = '\n'.join(lines[3:]).strip()


            # Summarize the email body
            chat_session = model.start_chat(history=[])
            summary_response = chat_session.send_message(f"Summarize this email: {body}")
            summarized_body = summary_response.text.strip()

            # Translate the summarized body to French
            translation_session = model.start_chat(history=[])
            translate_response = translation_session.send_message(f"Translate this to Marathi: {summarized_body}")
            translated_body = translate_response.text.strip()

            # Write the results to CSV
            writer.writerow({
                'sender': sender,
                'receiver': receiver,
                'summarized_body': summarized_body,
                'translated_body': translated_body
            })

In [6]:
def main():
    process_emails('emails.txt', 'emails_output.csv')

if __name__ == "__main__":
    main()